In [2]:
import discord
import os
import requests
import json
import random
from datetime import date
from nba_api.live.nba.endpoints import scoreboard
from ML import Game_Res

from text_service import textCLI
from dotenv import load_dotenv

load_dotenv()

MONGO_FLASK_ENDPOINT_ROOT = os.getenv("MONGO_FLASK_ENDPOINT_ROOT") 
MAIN_MONGO_API = os.getenv("MAIN_MONGO_API")
DATE_MONGO_API = os.getenv("DATE_MONGO_API")


In [3]:
def cleanGames(data):
    games = []
    today = date.today()
    for game in data:
            games.append(
                {"gameId":game["gameId"], 
                "home":{
                    "teamAbrev" : game['homeTeam']['teamTricode'],
                    "teamId":game['homeTeam']['teamId']
                    },
                "away":{
                    "teamAbrev" : game['awayTeam']['teamTricode'],
                    "teamId":game['awayTeam']['teamId']
                    }
                    })
    games = {
        "games" :games,
        "date" : today.strftime("%m/%d/%y")
        }
    return games 


In [4]:
def clean_pred_res(data):
    games = []
    today = date.today()
    for game in data:
        games.append({"home":{
                    "teamAbrev" : game['Home'],
                    "teamScore":game['HomeScore']
                    },
                "away":{
                    "teamAbrev" : game['Away'],
                    "teamScore":game['AwayScore']
                    }
        })
    return { 
        "games" :games,
        "date" : today.strftime("%m/%d/%y")
        }



In [4]:
def get_user_numbers():
    # url = "http://localhost:5000/mongodb"
    url = MONGO_FLASK_ENDPOINT_ROOT+MAIN_MONGO_API
    payload = json.dumps({
    "database": "nba_test",
    "collection": "user"
    })
    headers = {
    'Content-Type': 'application/json'
    }
    response = requests.request("GET", url, headers=headers, data=payload).json()
    return response

In [5]:
def sendTexts(game):
    users = get_user_numbers()
    responses = []
    for res in users:
        if res["sendText"] == "True":
            cli = textCLI()
            name = res["name"]
            home = game["Home"]
            away = game["Away"]
            homeScore = game["HomeScore"]
            awayScore = game["AwayScore"]
            message = f"\n {name}, \n Here are your predictions for {home} vs {away}: \n {home} : { homeScore } { away} : {awayScore}"
            messageRes = cli.sendMessage(str(res["phone"]),str(message))
            responses.append(messageRes)
        
    return (responses)

In [6]:
def update_games_db(gameData):
    if len(gameData) > 0:
        url = MONGO_FLASK_ENDPOINT_ROOT+MAIN_MONGO_API
        json_data = {"database": "nba_test", "collection": "game", "Document":gameData}
        headers = {'Content-type':'application/json', 'Accept':'application/json'}
        response = requests.post(url, json= json_data, headers=headers)
        print(response)



In [7]:
def update_predictions_db(predictions):
    if len(predictions) > 0:
        url = MONGO_FLASK_ENDPOINT_ROOT+MAIN_MONGO_API
        json_data = {"database": "nba_test", "collection": "predictions", "Document":predictions}
        headers = {'Content-type':'application/json', 'Accept':'application/json'}
        response = requests.post(url, json= json_data, headers=headers)
        print(response)

In [8]:
def get_games(dbFormat):
    board = scoreboard.ScoreBoard()
    print("ScoreBoardDate: " + board.score_board_date)
    games = board.games.get_dict()
    db_format = cleanGames(games)
    ml_format =db_format["games"]
    if dbFormat:
        return db_format
    return ml_format
get_games(True)

ScoreBoardDate: 2021-12-06


{'games': [{'gameId': '0022100353',
   'home': {'teamAbrev': 'CHA', 'teamId': 1610612766},
   'away': {'teamAbrev': 'PHI', 'teamId': 1610612755}},
  {'gameId': '0022100354',
   'home': {'teamAbrev': 'DET', 'teamId': 1610612765},
   'away': {'teamAbrev': 'OKC', 'teamId': 1610612760}},
  {'gameId': '0022100355',
   'home': {'teamAbrev': 'IND', 'teamId': 1610612754},
   'away': {'teamAbrev': 'WAS', 'teamId': 1610612764}},
  {'gameId': '0022100356',
   'home': {'teamAbrev': 'MIA', 'teamId': 1610612748},
   'away': {'teamAbrev': 'MEM', 'teamId': 1610612763}},
  {'gameId': '0022100357',
   'home': {'teamAbrev': 'CHI', 'teamId': 1610612741},
   'away': {'teamAbrev': 'DEN', 'teamId': 1610612743}},
  {'gameId': '0022100358',
   'home': {'teamAbrev': 'MIL', 'teamId': 1610612749},
   'away': {'teamAbrev': 'CLE', 'teamId': 1610612739}},
  {'gameId': '0022100359',
   'home': {'teamAbrev': 'MIN', 'teamId': 1610612750},
   'away': {'teamAbrev': 'ATL', 'teamId': 1610612737}},
  {'gameId': '0022100360'

In [9]:
def get_game_prediction(game):
    home = game["home"]["teamAbrev"]
    away = game["away"]["teamAbrev"]
    res = Game_Res(home,away)
    return res

In [13]:
def run_morning_ml():
    today = date.today()
    games = get_games(False)
    predictionResults = []
    completedFlag = False
    for i in range(len(games)):
        res = get_game_prediction(games[i])
        print(res)
        predictionResults.append(res)
    completedFlag =True
    print(predictionResults)
    if completedFlag ==True:
        update_predictions_db({"games":predictionResults,
        "date": today.strftime("%m/%d/%y") })
        
run_morning_ml()

ScoreBoardDate: 2021-12-06
2020 2021
-------------------------------------------------
2020-21
processing game 0 out of 72 for the 2020-21 season
processing game 10 out of 72 for the 2020-21 season
processing game 20 out of 72 for the 2020-21 season
processing game 30 out of 72 for the 2020-21 season
processing game 40 out of 72 for the 2020-21 season
processing game 50 out of 72 for the 2020-21 season
processing game 60 out of 72 for the 2020-21 season
processing game 70 out of 72 for the 2020-21 season
2020 2021
-------------------------------------------------
2020-21
processing game 0 out of 72 for the 2020-21 season
processing game 10 out of 72 for the 2020-21 season
processing game 20 out of 72 for the 2020-21 season
processing game 30 out of 72 for the 2020-21 season
processing game 40 out of 72 for the 2020-21 season
processing game 50 out of 72 for the 2020-21 season
processing game 60 out of 72 for the 2020-21 season
processing game 70 out of 72 for the 2020-21 season
Epoch 1

In [11]:
def send_todays_pred():
    games = get_predictions_today()
    today_games = games[0]['games'] 
    textRes = []
    for i in range(len(today_games)):
        text = sendTexts(today_games[i])
        textRes.append(text)
    print(textRes)


In [12]:
def get_predictions_today():
    url = MONGO_FLASK_ENDPOINT_ROOT+DATE_MONGO_API
    payload = json.dumps({
    "database": "nba_test",
    "collection": "predictions"
    })
    headers = {
    'Content-Type': 'application/json'
    }
    response = requests.request("GET", url, headers=headers, data=payload).json()
    return response
